We write all our import

In [1]:
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import imageio
import numpy as np
import os
import math
import tensorflow as tf
from scipy.signal import convolve2d
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import normalizers

Part with the local contrast normalisation

Work with LCN: What is the Best Multi-Stage Architecture for Object Recognition? http://yann.lecun.com/exdb/publis/pdf/jarrett-iccv-09.pdf

https://cedar.buffalo.edu/~srihari/CSE676/12.2%20Computer%20Vision.pdf

In [2]:
def LocalNormalization(img, P=3, Q=3, C=1):
    kernel = np.ones((P, Q)) / (P * Q)
    img_mean = convolve2d(img, kernel, boundary='symm', mode='same')
    img_sm = convolve2d(np.square(img), kernel, boundary='symm', mode='same')
    img_std = np.sqrt(np.maximum(img_sm - np.square(img_mean), 0)) + C
    img_ln = (img - img_mean) / img_std
    return img_ln

New part with the local response normalization

First article : ImageNet Classification with Deep Convolutional
Neural Networks http://www.cs.toronto.edu/~fritz/absps/imagenet.pdf

Second article : 
https://towardsdatascience.com/difference-between-local-response-normalization-and-batch-normalization-272308c034ac

Third Paper: https://prateekvjoshi.com/2016/04/05/what-is-local-response-normalization-in-convolutional-neural-networks/

En noir et blanc

In [ ]:
LRN = torch.nn.LocalResponseNorm(3)
def LRNUncolored(img):
  image = np.asarray(img)
  image_tensor = torch.Tensor(image)
  image_tensor = image_tensor.unsqueeze_(0)
  image_lrn = LRN(image_tensor).numpy().transpose((1,2,0))
  image_lrn = image_lrn.squeeze()
  scaled_img = (image_lrn - image_lrn.min())/(image_lrn.max() - image_lrn.min())
  return scaled_img

def LRNColored(img):
  image = np.asarray(img)
  image = image.transpose((2,0,1))
  image_tensor = torch.Tensor(image)
  image_lrn = LRN(image_tensor).numpy().transpose((1,2,0))
  scaled_img = (image_lrn - image_lrn.min())/(image_lrn.max() - image_lrn.min())
  return Image.fromarray(np.uint8(scaled_img * 255) , 'RGB')

* ZCA Whitening Normalization

In [3]:
def ZCA_whitening(img):  # WORKS W/ WARNINGS : img_cn.astype(np.uint8)
  normalizer = normalizers.Tools()

  # ** 1st - Apply contrast normalization to the image
  img_cn = normalizer.contrast_normalize(img)
  # ** 2nd - Apply ZCA sphering normalization to the image
  img_normalized = normalizer.ZCA_sphere(img_cn)

  return img_normalized

In [5]:
#nom des bases : TID2013 et CSIQ
#noms des normalization : LRN, LCN, ZCA et LFS
#Dossier 1 : TID2013_patch
#Dossier 2 : TID2013_LRN
#Dossier 3 : TID2013_LCN
#Dossier 4 : TID2013_ZCA
#Dossier 5 : TID2013_LFS
#Dossier 6 : CSIQ_LRN
#Dossier 7 : CSIQ_LCN
#Dossier 8 : CSIQ_ZCA
#Dossier 9 : CSIQ_LFS
#Dossier 10 : CSIQ_patch
def crop_img(chemin, img, bd1, bd2):
  imgC = Image.open(chemin + img)
  imgUC = Image.open(chemin + img).convert('L')
  longueur = imgC.size[0] / 64
  largeur = imgC.size[1] / 64
  for j in range(0,int(largeur)):
    for i in range(0,int(longueur)):
      patchC = imgC.crop((i * 64, j * 64, (i+1) * 64, (j+1) * 64))
      patch = imgUC.crop((i * 64, j * 64, (i+1) * 64, (j+1) * 64))
      current = str(i+j*longueur)
      
      #patch.save("/content/" + bd1 + "/" + "patch/" + img + "_patch_" + current + ".png", "PNG")
      #patch.save("/content/" + bd2 + "/" + "patch/" + img + "_patch_" + current + ".png", "PNG")

      #normalisation 1 avec ce patch pour TID2013
      #Image.fromarray(LocalNormalization(patch)).convert('L').save("/content/" + bd1 + "/" + "LCN/" + img + "_patch_" + current + ".png", "PNG")
      #normalisation 1 avec ce patch pour CSIQ
      #Image.fromarray(LocalNormalization(patch)).convert('L').save("/content/" + bd2 + "/" + "LCN/" + img + "_patch_" + current + ".png", "PNG")
      
      #normalisation 2 avec ce patch pour TID2013
      #LRNColored(patchC).save("/content/" + bd1 + "/" + "LRN/" + img + "_patch_" + current + ".png", "PNG")
      #normalisation 2 avec ce patch pour CSIQ
      #LRNColored(patchC).save("/content/" + bd2 + "/" + "LRN/" + img + "_patch_" + current + ".png", "PNG")

      #normalisation 3 avec ce patch pour TID2013

      #normalisation 3 avec ce patch pour CSIQ

      #normalisation 4 avec ce patch pour TID2013
      #imageio.imwrite("/content/" + bd1 + "/" + "ZCA/" + img + "_patch_" + current + ".png", ZCA_whitening(patch))
      #normalisation 4 avec ce patch pour CSIQ
      imageio.imwrite("/content/" + bd2 + "/" + "ZCA/" + img + "_patch_" + current + ".png", ZCA_whitening(patch))
      

In [4]:
#crée les deux dossiers des bds
#Crée les 5 sous dossiers dans chaque bd
import os.path
from os import path
def init_collab(bd1, bd2):
  if path.exists('/content/' + bd1) == False:
    os.mkdir('/content/' + bd1)
  if path.exists('/content/' + bd2) == False:
    os.mkdir('/content/' + bd2)
  if path.exists('/content/' + bd1 + '/patch') == False:
    os.mkdir('/content/' + bd1 + '/patch')
  if path.exists('/content/' + bd1 + '/LRN') == False:
    os.mkdir('/content/' + bd1 + '/LRN')
  if path.exists('/content/' + bd1 + '/LCN') == False:
    os.mkdir('/content/' + bd1 + '/LCN')
  if path.exists('/content/' + bd1 + '/ZCA') == False:
    os.mkdir('/content/' + bd1 + '/ZCA')
  if path.exists('/content/' + bd1 + '/LFS') == False:
    os.mkdir('/content/' + bd1 + '/LFS')
  if path.exists('/content/' + bd2 + '/patch') == False:
    os.mkdir('/content/' + bd2 + '/patch')
  if path.exists('/content/' + bd2 + '/LRN') == False:
    os.mkdir('/content/' + bd2 + '/LRN')
  if path.exists('/content/' + bd2 + '/LCN') == False:
    os.mkdir('/content/' + bd2 + '/LCN')
  if path.exists('/content/' + bd2 + '/ZCA') == False:
    os.mkdir('/content/' + bd2 + '/ZCA')
  if path.exists('/content/' + bd2 + '/LFS') == False:
    os.mkdir('/content/' + bd2 + '/LFS')

In [6]:
def download_zip():
  #zip CSIQ and TID2013
  !zip -r "/content/CSIQ.zip" "/content/CSIQ"
  !zip -r "/content/TID2013.zip" "/content/TID2013"
  
  #function to download all the new png images in zip
  from google.colab import files
  files.download("/content/CSIQ.zip")
  files.download("/content/TID2013.zip")

In [7]:
#init collab
bd1 = 'CSIQ'
bd2 = 'TID2013'
init_collab(bd1, bd2)

In [8]:
# REMI
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# HENINTSOA
from google.colab import drive
drive.mount('/gdrive/')

In [9]:
if path.exists('/content/origin_TID2013') == False:
    os.mkdir('/content/origin_TID2013')
if path.exists('/content/origin_CSIQ') == False:
    os.mkdir('/content/origin_CSIQ')

Create folder for origin database that are in my drive

In [11]:
# REMI

!unzip '/content/drive/MyDrive/Master/Research lab/TID2013.zip' -d '/content/origin_TID2013'
#!unzip '/content/drive/MyDrive/Master/Research lab/CSIQ.zip' -d '/content/origin_CSIQ'

Archive:  /content/drive/MyDrive/Master/Research lab/TID2013.zip
   creating: /content/origin_TID2013/TID2013/
  inflating: /content/origin_TID2013/TID2013/i23_04_2.bmp  
  inflating: /content/origin_TID2013/TID2013/i20_23_2.bmp  
  inflating: /content/origin_TID2013/TID2013/i16_23_4.bmp  
  inflating: /content/origin_TID2013/TID2013/i11_12_4.bmp  
  inflating: /content/origin_TID2013/TID2013/i20_06_2.bmp  
  inflating: /content/origin_TID2013/TID2013/i23_15_2.bmp  
  inflating: /content/origin_TID2013/TID2013/i01_22_4.bmp  
  inflating: /content/origin_TID2013/TID2013/i23_14_1.bmp  
  inflating: /content/origin_TID2013/TID2013/i01_01_4.bmp  
  inflating: /content/origin_TID2013/TID2013/i01_04_1.bmp  
  inflating: /content/origin_TID2013/TID2013/i17_16_2.bmp  
  inflating: /content/origin_TID2013/TID2013/i21_18_3.bmp  
  inflating: /content/origin_TID2013/TID2013/i04_04_3.bmp  
  inflating: /content/origin_TID2013/TID2013/i20_24_4.bmp  
  inflating: /content/origin_TID2013/TID2013/i15_

In [ ]:
# HENINTSOA
!unzip '/gdrive/MyDrive/Documents/Etudes/M1/EUR/Research Lab/Data/TID2013.zip' -d '/content/origin_TID2013'
!unzip '/gdrive/MyDrive/Documents/Etudes/M1/EUR/Research Lab/Data/CSIQ.zip' -d '/content/origin_CSIQ'

unzip:  cannot find or open /gdrive/MyDrive/Documents/Etudes/M1/EUR/Research Lab/Data/TID2013.zip, /gdrive/MyDrive/Documents/Etudes/M1/EUR/Research Lab/Data/TID2013.zip.zip or /gdrive/MyDrive/Documents/Etudes/M1/EUR/Research Lab/Data/TID2013.zip.ZIP.
unzip:  cannot find or open /gdrive/MyDrive/Documents/Etudes/M1/EUR/Research Lab/Data/CSIQ.zip, /gdrive/MyDrive/Documents/Etudes/M1/EUR/Research Lab/Data/CSIQ.zip.zip or /gdrive/MyDrive/Documents/Etudes/M1/EUR/Research Lab/Data/CSIQ.zip.ZIP.


In [12]:
#take all bmp images in CSIQ
#images  = [f for f in os.listdir('/content/origin_CSIQ/CSIQ/dst_imgs/') if os.path.splitext(f)[-1] == '.png' or os.path.splitext(f)[-1] == '.PNG']
#for img in (images):
 # crop_img('/content/origin_CSIQ/CSIQ/dst_imgs/', img, bd1, bd2) ##launch the crop on this img

#take all bmp images in TDI2013
images  = [f for f in os.listdir('/content/origin_TID2013/TID2013/') if os.path.splitext(f)[-1] == '.bmp' or os.path.splitext(f)[-1] == '.BMP']
for img in (images):
  crop_img('/content/origin_TID2013/TID2013/', img, bd1, bd2) ##launch the crop on this img

#download_zip()

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.


Move zip to my drive

In [13]:
# REMI
#zip CSIQ and TID2013
!zip -r "/content/TID2013_ZCA.zip" "/content/TID2013"
#!mv "/content/CSIQ_ZCA.zip" "/content/drive/MyDrive/Master/Research lab"
!mv "/content/TID2013_ZCA.zip" "/content/drive/MyDrive/Master/Research lab"

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  adding: content/TID2013/ZCA/i17_05_3.bmp_patch_36.0.png (stored 0%)
  adding: content/TID2013/ZCA/i13_23_3.bmp_patch_36.0.png (stored 0%)
  adding: content/TID2013/ZCA/I07_01_1.bmp_patch_23.0.png (stored 0%)
  adding: content/TID2013/ZCA/i17_20_4.bmp_patch_29.0.png (stored 0%)
  adding: content/TID2013/ZCA/i02_19_4.bmp_patch_8.0.png (stored 0%)
  adding: content/TID2013/ZCA/i05_04_5.bmp_patch_35.0.png (stored 0%)
  adding: content/TID2013/ZCA/i11_11_3.bmp_patch_36.0.png (stored 0%)
  adding: content/TID2013/ZCA/i09_16_1.bmp_patch_0.0.png (stored 0%)
  adding: content/TID2013/ZCA/I11_13_1.BMP_patch_4.0.png (stored 0%)
  adding: content/TID2013/ZCA/i13_08_2.bmp_patch_42.0.png (stored 0%)
  adding: content/TID2013/ZCA/i19_04_4.bmp_patch_43.0.png (stored 0%)
  adding: content/TID2013/ZCA/i03_19_5.bmp_patch_13.0.png (stored 0%)
  adding: content/TID2013/ZCA/i07_08_2.bmp_patch_43.0.png (stored 0%)
  adding: conte

In [ ]:
# HENINTSOA
!mv "/content/CSIQ_normalized.zip" "/gdrive/MyDrive/Documents/Etudes/M1/EUR/Research Lab/Data"
!mv "/content/TID2013_normalized.zip" "/gdrive/MyDrive/Documents/Etudes/M1/EUR/Research Lab/Data"